# Known Problem:
- **Unresolved**
    1. how to solve for eigen vectors
- **Partially Resolved**
    1. Certain websites cause the error of, specified here: https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests, which is a connection error. 
        - this appears to be windows specific as this was not a relevant issue when testing code in ubuntu 22.04, some forum reflect similar message
        - FAILED SOLUTION: first response in that stackoverflow page, still kept the stuff bc it seemed like better QOL
        - ATTEMPTED SOLUTION: used try and except. NOTE: this solution is not ideal as it does not resolve the issue, it just ignores it and moves on.
- **Resolved**
    1. The search "what is a taylor series" returns SSLCertVerification Error.
        - this resolution does not consider the security of the websites being accessed, potentially use try except to have the request verify when possible and not break when cannot verify
        - ATTEMPTED SOLUTION: used try except to use verify=False when ssl error occurs. Nested try except to ensure that both connection error and ssl error are handled.

    

    

# Part 1: Data Acquisition & Preparation

## Collecting & Preprocessing Our Data

In [2]:
#imports
import requests #send HTTP requests to web pages and retrieve their content
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from urllib.parse import urlparse, urljoin
import pandas as pd
from nltk.corpus import stopwords
import http.client
import re

In [3]:
import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
#uncomment if you cannot properly download punkt

nltk.download('punkt')
#ssl._create_default_https_context = ssl._create_default_https_context
#uncomment if you cannot properly download punkt

#remove filler words from an example column
nltk.download("stopwords")

nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\William
[nltk_data]     Zhang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\William
[nltk_data]     Zhang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\William
[nltk_data]     Zhang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import concurrent.futures
from tqdm import tqdm
#Working implementation

session = requests.Session()
retry = Retry(connect=2, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)
# Define the search query and retrieve search results
search_query = input("What would you like to search up?")
search_engine_url = "https://www.google.com/search?q="

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Number of search result pages to retrieve (each page typically has 10 results)
num_pages = 5

search_results = []

def verify_ssl_certificate(url):
    try:
        response = session.get(url, verify=True, headers=headers)
        return True
    except requests.exceptions.SSLError:
        return False
    except Exception as e:
        print("Error:", e)
        return False

for page in range(num_pages):
    # Calculate the start index for the current page
    start_index = page * 10

    try:
        # Make a GET request to retrieve the search result page
        response = session.get(search_engine_url + search_query + f"&start={start_index}", headers=headers)
        response.raise_for_status()  # Raise an exception for non-2xx status codes

    except requests.exceptions.RequestException as e:
        print("Error occurred for page", page+1)
        print("Exception:", e)
        continue

    # Parse and extract information from the search result page
    soup = BeautifulSoup(response.content, "html.parser")

    for result in soup.select("div.g"):
        title_element = result.select_one("h3")
        if title_element:
            title = title_element.get_text()
            url_element = result.find("a")
            if url_element:
                url = url_element["href"]
                parsed_url = urlparse(url)
                if parsed_url.scheme == "":
                    # Relative URL, convert it to absolute URL
                    base_url = response.url
                    url = urljoin(base_url, url)

                # Verify SSL certificate before making the GET request
                if verify_ssl_certificate(url):
                    try:
                        print("Trying:", url)
                        result_response = session.get(url, headers=headers)
                        result_response.raise_for_status()  # Raise an exception for non-2xx status codes
                        result_soup = BeautifulSoup(result_response.content, "html.parser")
                        text = result_soup.get_text()
                        text = ' '.join(text.split())
                        search_results.append({"Title": title, "URL": url, "Text": text})
                    except http.client.IncompleteRead:
                        print("Error: Incomplete read for:", url)
                        continue
                    except Exception as e:
                        print("Error: Failed to retrieve content for:", url)
                        print("Exception:", e)
                        continue

                # Make a GET request to the URL of the search result
                else:
                    print("SSL certificate verification failed for:", url)
                    continue
                    

print("Grabbed search results")

# Process the search results as desired

Trying: https://www.ibm.com/topics/deep-learning
Trying: https://www.mathworks.com/discovery/deep-learning.html
Trying: https://aws.amazon.com/what-is/deep-learning/
Error: HTTPSConnectionPool(host='machinelearningmastery.com', port=443): Max retries exceeded with url: /what-is-deep-learning/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E09F933B50>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
SSL certificate verification failed for: https://machinelearningmastery.com/what-is-deep-learning/
Trying: https://en.wikipedia.org/wiki/Deep_learning
Trying: https://www.simplilearn.com/tutorials/deep-learning-tutorial/what-is-deep-learning
Trying: https://www.oracle.com/artificial-intelligence/machine-learning/what-is-deep-learning/
Trying: https://www.geeksforgeeks.org/introduction-deep-learning/
Trying: https://www.techtarget.com/searchenterpriseai/definition/deep-learning-deep-neural-network
Trying: https://builtin.com/mach

In [5]:
df = pd.DataFrame(search_results)
df_bart = pd.DataFrame(search_results)
df_bart_unlem = pd.DataFrame(search_results)
# Extract domain names from URLs and set as the index for the dataframe
tld_pattern = re.compile(r'\.[a-zA-Z]+$')
df['Domain'] = df['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df.insert(0, 'Domain', df.pop('Domain'))

df_bart['Domain'] = df_bart['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df_bart.insert(0, 'Domain', df_bart.pop('Domain'))
#Handles the "Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER" exception
df_bart['Text'] = df_bart['Text'].str.replace('\ufffd', '')

df_bart_unlem['Domain'] = df_bart_unlem['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df_bart_unlem.insert(0, 'Domain', df_bart_unlem.pop('Domain'))
#Handles the "Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER" exception
df_bart_unlem['Text'] = df_bart_unlem['Text'].str.replace('\ufffd', '')

#unfiltered dataframe
print(len(df))
df.head()

42


,Domain,Title,URL,Text
0,ibm,What is Deep Learning?,https://www.ibm.com/topics/deep-learning,What is Deep Learning? | IBM What is deep lear...
1,mathworks,"What Is Deep Learning? | How It Works, Techniq...",https://www.mathworks.com/discovery/deep-learn...,"What Is Deep Learning? | How It Works, Techniq..."
2,aws.amazon,What is Deep Learning?,https://aws.amazon.com/what-is/deep-learning/,What is Deep Learning? - Deep Learning Explain...
3,en.wikipedia,Deep learning,https://en.wikipedia.org/wiki/Deep_learning,Deep learning - Wikipedia Jump to content Main...
4,simplilearn,What is Deep Learning and How Does It Works [E...,https://www.simplilearn.com/tutorials/deep-lea...,What is Deep Learning and How Does It Works [U...


In [6]:
stop_words = stopwords.words("english") #english filler words

def preprocess(text):
    '''
    Input a text block and filter out unneeded characters
    returns a filtered text block in the form of a str
    Function filters whitespace, numbers, special characters, stopwords; handles case normalization 
    '''
    # Remove special characters and numbers
    cleaned_text = re.sub(r"[^a-zA-Z]+", " ", text)
    
    # Convert to lowercase and split into words
    words = cleaned_text.lower().split()
    
    # Remove stop words and single-character words
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1]
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in filtered_words])
    
    return lemmatized_text

def bart_preprocess(text):

    words = text.split()
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in words])

    return lemmatized_text



In [7]:
example = df_bart["Text"][0]
processed_example = bart_preprocess(example)

print("length of unfiltered text: ", len(example))
print("length of filtered text: ", len(processed_example))

length of unfiltered text:  9684
length of filtered text:  9547


In [8]:
#Properly using the apply() function onto a pandas column
#removes both filler words and special characters
df["Text"] = df["Text"].apply(preprocess)
df_bart["Text"] = df_bart["Text"].apply(bart_preprocess)


In [9]:
print(len(df_bart["Text"][0]))

9547


# NOTE
Lemmatization doesn't seem to remove much here

In [10]:
#filtered text for first row entry
df_bart["Text"][0]

'What is Deep Learning? | IBM What is deep learning? Deep learning enables system to cluster data and make prediction with incredible accuracy Subscribe to the IBM newsletter What is deep learning? Deep learning is a subset of machine learning, which is essentially a neural network with three or more layers. These neural network attempt to simulate the behavior of the human brain—albeit far from matching it ability—allowing it to “learn” from large amount of data. While a neural network with a single layer can still make approximate predictions, additional hidden layer can help to optimize and refine for accuracy. Deep learning drive many artificial intelligence (AI) application and service that improve automation, performing analytical and physical task without human intervention. Deep learning technology lie behind everyday product and service (such a digital assistants, voice-enabled TV remotes, and credit card fraud detection) a well a emerging technology (such a self-driving cars)

# Part 2: EDA, Feature Engineering, and Model Development

### Future Optimizations

1. Ignore website navigation for websites like (wikipedia, stackoverflow, etc.)


### Tasks

1. Research into transformer, BART model, to better understand model set up and how to get highest performance
2. Experiment with BART input (lemmatize/not, capitalization, etc.) to find best performance 
3. Try to implement BART model with recursion as per the drawn diagram 
4. Research into paralellization of transformer models with HuggingFace 

### Notes

1. To actually make project work, we need to create a few metrics that we can train a DL model on. Maybe, linguistic complexity (formulaic), sentiment analysis, text length, etc.

In [11]:
import torch
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame called 'df' containing your data
# Split the data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Split the train data further into train and validation sets (80% train, 20% validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Print the sizes of each dataset
print("Train set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))

Train set size: 26
Validation set size: 7
Test set size: 9


In [12]:
test_data.reset_index().drop(["index"], axis = 1)
val_data.reset_index().drop(["index"], axis = 1)
train_data.reset_index().drop(["index"], axis = 1)

,Domain,Title,URL,Text
0,en.wikipedia,Deep learning,https://en.wikipedia.org/wiki/Deep_learning,deep learning wikipedia jump content main menu...
1,oracle,What is Deep Learning?,https://www.oracle.com/artificial-intelligence...,deep learning oracle accessibility policy skip...
2,geeksforgeeks,Introduction to Deep Learning,https://www.geeksforgeeks.org/introduction-dee...,introduction deep learning geeksforgeeks skip ...
3,netapp,What Is Deep Learning? - How It Works,https://www.netapp.com/artificial-intelligence...,deep learning work netapp bluexp login netapp ...
4,meltwater,The Fundamentals of Deep Learning,https://www.meltwater.com/en/blog/fundamentals...,fundamental deep learningour suiteby needresou...
5,tibco,What is Deep Learning?,https://www.tibco.com/reference-center/what-is...,deep learning tibco software skip main content...
6,azure.microsoft,What Is Deep Learning?,https://azure.microsoft.com/en-us/resources/cl...,deep learning microsoft azure browser longer s...
7,databricks,What is Deep Learning?,https://www.databricks.com/glossary/deep-learning,deep learning platformthe databricks lakehouse...
8,flatironschool,What Is Deep Learning With Python?,https://flatironschool.com/blog/what-is-deep-l...,deep learning python flatiron school skip cont...
9,eweek,What Is Deep Learning?,https://www.eweek.com/enterprise-apps/deep-lea...,deep learning eweek close latest news artifici...


In [13]:
#Using Bart
# !pip install transformers
# !pip install torch
import torch
from transformers import BartForConditionalGeneration, AutoTokenizer

c:\Users\William Zhang\.conda\envs\search_sense\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")


NameError: name 'BartForConditionalGeneration' is not defined

In [ ]:
# len(df_bart["Text"][0])

41231

In [ ]:
# inputs = tokenizer([df_bart["Text"][2]],max_length = 1024,return_tensors="pt")


In [ ]:
summary_ids = model.generate(inputs["input_ids"], num_beams=1, min_length=0, max_length=30)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'What is RGB Color? – Nix Sensor Ltd – NIX Sensor Ltd Skip to content BRAND NEW Nix Mini 3'

Testing lemmatize or not

In [14]:
from transformers import AutoTokenizer, LEDForConditionalGeneration

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv")
tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384-arxiv")


LEMMATIZED

In [15]:
inputs = tokenizer.encode(df_bart["Text"][1], return_tensors="pt")

global_attention_mask = torch.zeros_like(inputs)
global_attention_mask[:, 0] = 1

model = model.to(device)
inputs = inputs.to(device)
global_attention_mask = global_attention_mask.to(device)

summary_ids = model.generate(inputs, global_attention_mask=global_attention_mask, num_beams = 1, max_length = 64)
print(tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
print(df_bart["Text"][1])

 deep learning is a machine learning technique that teach computer to do what come naturally to humans: learn by example. in deep learning, a computer model learns to perform classification task directly from images, text, or sound. deep learning is a specialized form of machine learning. 
 deep learning achieves recognition accuracy at higher level
What Is Deep Learning? | How It Works, Techniques & Applications - MATLAB & Simulink Skip to content Toggle Main Navigation Sign In to Your MathWorks Account My Account My Community Profile Link License Sign Out Products Solutions Academia Support Community Events Get MATLAB MATLAB Products Solutions Academia Support Community Events Get MATLAB MATLAB Sign In to Your MathWorks Account My Account My Community Profile Link License Sign Out Deep Learning Search MathWorks.com Search Close Mobile Search Close Mobile Search What Is Deep Learning? 3 thing you need to know Deep learning is a machine learning technique that teach computer to do what

NOT LEMMATIZED

In [31]:
inputs = tokenizer.encode(df_bart_unlem["Text"][1], return_tensors="pt")

global_attention_mask = torch.zeros_like(inputs)
global_attention_mask[:, 0] = 1

model = model.to(device)
inputs = inputs.to(device)
global_attention_mask = global_attention_mask.to(device)

summary_ids = model.generate(inputs, global_attention_mask=global_attention_mask, num_beams = 1, max_length = 64)
print(tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
print("____________________________________________")
print(df_bart_unlem["Text"][1])

 the aim of this article is to provide a concise introduction to the subject of determining the eigenvectors of a matrix. 
 the purpose of this article is to provide a concise introduction to the subject of determining the eigenvectors of a matrix. 
 the purpose of this article is to provide a
____________________________________________
Determining the Eigenvectors of a Matrix My Preferences My Reading List Literature Notes Test Prep Study Guides Log In Sign Up Linear Algebra ! Home Study Guides Linear Algebra Determining the Eigenvectors of a Matrix All Subjects Vector Algebra The SpaceR2 The SpaceR3 The SpaceRn Matrix Algebra Matrices Operations with Matrices Linear Systems Gaussian Elimination Using Elementary Row Operations to Determine A−1 Introduction to Linear Systems Solutions to Linear Systems Real Euclidean Vector Spaces Linear Combinations and Span Linear Independence The Rank of a Matrix A Basis for a Vector Space Projection onto a Subspace Row Space and Column Space of a 

# NOTES ABOUT SUMMARIZATION
1. We need to filter out the navigation content for Bart to do a good job, I assume we can accomplish that through briefly studying the html of some of the web pages
2. Some website includes latex to describe mathematical expressions or other such expressions, hence BART cannot read that and cannot comprehend that, we either need to integrate a latex parser or filter out those expressions
3. Research why LED keeps cutting off some of the summary, how to fix this



4. The model is trained on a specific domain (arxiv) and will not perform well on other domains (written by copilot not sure if valid)
